# Experiment 3, Model 1

#### Model Setup

Run models in the following order, using their output labels as features for the next model:

1. Multiclass Person Name + Occupation Sequence Classifier

2. Multilabel Stereotype + Omission Document Classifier

***

* Supervised learning
    * Train, Validate, and (Blind) Test Data: under directory `../data/token_clf_data/experiment_input/`
    * Prediction Data: Data: under directory `../data/token_clf_data/model_output/experiment3/`
* Word Embeddings
    * Custom fastText (word2vec with subwords) embeddings of 100 dimensions trained on the CRC Archives catalog's descriptive metadata (harvested October 2020)

***

**Table of Contents**

[I.](#i) Person Name + Occupation Sequence Classifier
* [Preprocessing](#prep)
* [Training & Prediction](#tp)
* [Evaluation](#eval)

Load resources:

In [1]:
# For custom functions and variables
import utils, utils1, config

# For data analysis
import pandas as pd
import numpy as np
import os, re

# For creating directories
from pathlib import Path

# For preprocessing
from gensim.models import FastText
from gensim import utils as gensim_utils

# For multilabel token classification
import sklearn.metrics
from sklearn.preprocessing import MultiLabelBinarizer
from skmultilearn.problem_transform import ClassifierChain
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

# For multiclass sequence classification
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

Define resources for the models:

In [2]:
Path(config.experiment_input_path).mkdir(parents=True, exist_ok=True)    # For train, devtest, and blind test data
# Path(config.experiment1_output_path).mkdir(parents=True, exist_ok=True)  # For predictions
# Path(config.experiment1_agmt_path).mkdir(parents=True, exist_ok=True)    # For agreement metrics

predictions_dir = config.experiment3_path+"5fold/output"     # For predictions
Path(predictions_dir).mkdir(parents=True, exist_ok=True)
agreement_dir = config.experiment3_path+"5fold/agreement"    # For agreement metrics
Path(agreement_dir).mkdir(parents=True, exist_ok=True)

In [3]:
# Model 2:
pers_o_label_subset = ["B-Unknown", "I-Unknown", "B-Feminine", "I-Feminine", "B-Masculine", "I-Masculine", "B-Occupation", "I-Occupation"]

In [4]:
pers_o_label_tags = {
    "Unknown": ["B-Unknown", "I-Unknown"], "Feminine": ["B-Feminine", "I-Feminine"], "Masculine": ["B-Masculine", "I-Masculine"],
     "Occupation": ["B-Occupation", "I-Occupation"]
    }

In [5]:
d = 100  # dimensions of word embeddings (should match utils1.py)
target_labels = "pers_o"  # for file names

<a id="1"></a>
## 1. Person Name + Occupation Labels

Train a multiclass sequence classifier, using Conditional Random Field with Adaptive Regularization of Weight Vectors (AROW), on the Person Name and Occupation labels.

Multiclass is a suitable setup for these labels because they are mutually exclusive (no one token should have more than one of these labels).  The sequence classifier with AROW was the highest performing for past algorithm experiments with sequence classifiers for Person Name and Occupation labels.

The devtest data subset from the model in step 1 will be the train data subset in this step, with the predicted Linguistic labels as features passed into this second model.  The train data subset from the first model will be the devtest data subset for this second model.

In [6]:
### For 40-40-20 data split
# train_df = pd.read_csv(config.tokc_path+"experiment_input/token_validate.csv", index_col=0)
# dev_df =  pd.read_csv(config.tokc_path+"experiment_input/token_train.csv", index_col=0)
# perso_train, perso_dev = utils.selectDataForLabels(train_df, dev_df, "tag", pers_o_label_subset)
# print(perso_train.shape, perso_dev.shape)
# ------------------------
### For this experiment, we'll repeatedly train models on different 80% selections of 
### data and predict on the remaining 20% split, for a modified 5-fold cross-validation approach.
perso_df = pd.read_csv(config.tokc_path+"experiment_input/token_5fold.csv", index_col=0)
# Make sure only Person Name and Occupation tags are considered
perso_df = utils1.selectDataForLabels(perso_df, "tag", pers_o_label_subset)
perso_df.head()

,description_id,sentence_id,ann_id,token_id,token,token_offsets,pos,tag,field,fold
0,0,0,99999,0,Identifier,"(0, 10)",NN,O,Identifier,split4
1,0,0,99999,1,:,"(10, 11)",:,O,Identifier,split4
2,0,0,99999,2,AA5,"(12, 15)",NN,O,Identifier,split4
3,1,1,99999,3,Title,"(17, 22)",NN,O,Title,split2
4,1,1,99999,4,:,"(22, 23)",:,O,Title,split2


In [7]:
perso_df.tag.unique()  # Looks good

array(['O', 'B-Unknown', 'B-Masculine', 'I-Unknown', 'I-Masculine',
       'B-Occupation', 'I-Occupation', 'B-Feminine', 'I-Feminine'],
      dtype=object)

Define the five groups of training and test sets:

In [8]:
split_col = "fold"
splits = perso_df[split_col].unique()
splits.sort()
print(splits)
train0, test0 = list(splits[:4]), splits[4]
train1, test1 = list(splits[1:]), splits[0]
train2, test2 = list(splits[2:])+[splits[0]], splits[1]
train3, test3 = list(splits[3:])+list(splits[:2]), splits[2]
train4, test4 = [splits[4]]+list(splits[:3]), splits[3]
runs = [(train0, test0), (train1, test1), (train2, test2), (train3, test3), (train4, test4)]
for run in runs:
    print(run)

['split0' 'split1' 'split2' 'split3' 'split4']
(['split0', 'split1', 'split2', 'split3'], 'split4')
(['split1', 'split2', 'split3', 'split4'], 'split0')
(['split2', 'split3', 'split4', 'split0'], 'split1')
(['split3', 'split4', 'split0', 'split1'], 'split2')
(['split4', 'split0', 'split1', 'split2'], 'split3')


Looks good!

<a id="prep"></a>
#### Preprocessing

In [9]:
# train_df = perso_train.drop(columns=["description_id", "ann_id", "token_offsets", "field", "subset", "pos"])
# dev_df = perso_dev.drop(columns=["description_id", "ann_id", "token_offsets", "field", "subset", "pos"])
# ------------------------
df = perso_df.drop(columns=["description_id", "ann_id", "token_offsets", "field", "pos"])

In [10]:
# df_train_token_groups = utils.implodeDataFrame(train_df, ['token_id', 'sentence_id', 'token'])
# df_train_token_groups = df_train_token_groups.reset_index()
# # df_train_token_groups.head()
# df_dev_token_groups = utils.implodeDataFrame(dev_df, ['token_id', 'sentence_id', 'token'])
# df_dev_token_groups = df_dev_token_groups.reset_index()
# # df_dev_token_groups.head()
# df_train_grouped = utils.implodeDataFrame(df_train_token_groups, ['sentence_id'])
# df_dev_grouped = utils.implodeDataFrame(df_dev_token_groups, ['sentence_id'])
# df_train_grouped = df_train_grouped.rename(columns={"token":"sentence"})
# df_dev_grouped = df_dev_grouped.rename(columns={"token":"sentence"})
# df_train_grouped.head()
# ------------------------

In [11]:
df_token_groups = utils.implodeDataFrame(df, ['token_id', 'sentence_id', 'token', 'fold'])
df_token_groups = df_token_groups.reset_index()
# # df_token_groups.head()

In [12]:
df_grouped = utils.implodeDataFrame(df_token_groups, ['sentence_id', 'fold'])
df_grouped = df_grouped.rename(columns={"token":"sentence"})
df_grouped = df_grouped.reset_index()
df_grouped.head()

,sentence_id,fold,token_id,sentence,tag
0,0,split4,"[0, 1, 2]","[Identifier, :, AA5]","[[O], [O], [O]]"
1,1,split2,"[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]","[Title, :, Papers, of, The, Very, Rev, Prof, J...","[[O], [O], [O], [O], [O, B-Unknown, B-Masculin..."
2,2,split1,"[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 2...","[Scope, and, Contents, :, Sermons, and, addres...","[[O], [O], [O], [O], [O], [O], [O], [O], [O], ..."
3,3,split2,"[109, 110, 111, 112, 113, 114, 115, 116, 117, ...","[Biographical, /, Historical, :, Professor, Ja...","[[O], [O], [O], [O], [B-Masculine], [I-Masculi..."
4,4,split4,"[134, 135, 136, 137, 138, 139, 140, 141, 142, ...","[He, was, educated, at, Daniel, Stewart, 's, C...","[[O], [O], [O], [O], [O], [O], [O], [O], [O], ..."


Zip the linguistic label and BIO tags together with the tokens so each sentence item is a tuple: `(TOKEN, TAG_LIST)`

In [13]:
# df_train_grouped = df_train_grouped.reset_index()
# df_dev_grouped = df_dev_grouped.reset_index()
# train_sentences_pers = utils1.zip1Feature1AndTarget(df_train_grouped, "tag")  # Dev because not using additional feature col for linguistic labels
# print(train_sentences_pers[2][:3])
# dev_sentences_pers = utils1.zip1FeatureAndTarget(df_dev_grouped, "tag")
# print(dev_sentences_pers[0][:3])

In [14]:
# train_sentences = train_sentences_pers
# dev_sentences = dev_sentences_pers

In [15]:
# # Features
# X_train = [utils1.extractSentenceFeatures(sentence) for sentence in train_sentences]
# X_dev = [utils1.extractSentenceFeatures(sentence) for sentence in dev_sentences]
# # Target
# y_train = [utils1.extractSentenceTargets(sentence) for sentence in train_sentences]
# y_dev = [utils1.extractSentenceTargets(sentence) for sentence in dev_sentences]

<a id="tp"></a>
#### Training & Prediction

Train a Conditional Random Field (CRF) model with the default parameters on the **Person Name** category of tags.  We'll increase the max iterations to 100 for this model.

In [16]:
a = "arow"

In [17]:
# clf_pers = sklearn_crfsuite.CRF(algorithm=a, variance=0.5, max_iterations=100, all_possible_transitions=True)
# # https://stackoverflow.com/questions/66059532/attributeerror-crf-object-has-no-attribute-keep-tempfiles
# try:
#     clf_pers.fit(X_train, y_train)
# except AttributeError:
#     pass

In [18]:
pred_df = pd.DataFrame()

# Specify the run one at a time (with for loop, kernel dies; also, 
# crf_suite for sklearn's models will keep learning from previous runs if not restarted)
run = runs[0]  # 1, 2, 3, 4

# Get the train (80%) and test (20%) subsets of data
train_splits, test_split = run[0], run[1]
print("Training on:", train_splits)
train_df = df_grouped.loc[df_grouped[split_col].isin(train_splits)]
dev_df = df_grouped.loc[df_grouped[split_col] == test_split]

# Zip feature and target columns together so each 
# sentence item is a tuple: `(TOKEN, TAG_LIST)`
train_sentences = utils1.zip1FeatureAndTarget(train_df, "tag")
dev_sentences = utils1.zip1FeatureAndTarget(dev_df, "tag")
# Extract features
X_train = [utils1.extractSentenceFeatures(sentence) for sentence in train_sentences]
X_dev = [utils1.extractSentenceFeatures(sentence) for sentence in dev_sentences]
# Extract targets
y_train = [utils1.extractSentenceTargets(sentence) for sentence in train_sentences]
y_dev = [utils1.extractSentenceTargets(sentence) for sentence in dev_sentences]

# Train a classification model
clf_pers = sklearn_crfsuite.CRF(algorithm=a, variance=0.5, max_iterations=100, all_possible_transitions=True)
# https://stackoverflow.com/questions/66059532/attributeerror-crf-object-has-no-attribute-keep-tempfiles
try:
    clf_pers.fit(X_train, y_train)
except AttributeError:
    pass

# Predict with the trained model
print("Predicting on:", test_split)
predictions = clf_pers.predict(X_dev)
dev_df = dev_df.rename(columns={"tag":"tag_pers_o_expected"})
dev_df.insert(len(dev_df.columns), "tag_pers_o_predicted", predictions)
dev_df = dev_df.set_index(["sentence_id", "fold"])
dev_df_exploded = dev_df.explode(list(dev_df.columns))

if pred_df.shape[0] > 0:
    pred_df = pd.concat([pred_df, dev_df_exploded])
else:
    pred_df = dev_df_exploded

assert pred_df.loc[pred_df.tag_pers_o_predicted.isna()].shape[0] == 0, "Any NaN values should be replaced with 'O'"

filename = "crf_{a}_pers_o_baseline_fastText{d}_nolingfeatures_predictions_{s}.csv".format(a=a, t=target_labels, d=d, s=test_split)
pred_df.to_csv(predictions_dir+filename)

print("Predictions for {} saved!".format(test_split))

Training on: ['split0', 'split1', 'split2', 'split3']
Predicting on: split4
Predictions for split4 saved!


Combine the prediction data:

In [20]:
pred_df0 = pd.read_csv(predictions_dir+"crf_{a}_{t}_baseline_fastText{d}_nolingfeatures_predictions_split0.csv".format(a=a, t=target_labels, d=d), index_col=0)
pred_df1 = pd.read_csv(predictions_dir+"crf_{a}_{t}_baseline_fastText{d}_nolingfeatures_predictions_split1.csv".format(a=a, t=target_labels, d=d), index_col=0)
pred_df2 = pd.read_csv(predictions_dir+"crf_{a}_{t}_baseline_fastText{d}_nolingfeatures_predictions_split2.csv".format(a=a, t=target_labels, d=d), index_col=0)
pred_df3 = pd.read_csv(predictions_dir+"crf_{a}_{t}_baseline_fastText{d}_nolingfeatures_predictions_split3.csv".format(a=a, t=target_labels, d=d), index_col=0)
pred_df4 = pd.read_csv(predictions_dir+"crf_{a}_{t}_baseline_fastText{d}_nolingfeatures_predictions_split4.csv".format(a=a, t=target_labels, d=d), index_col=0)
pred_perso = pd.concat([pred_df0, pred_df1, pred_df2, pred_df3, pred_df4])
print(pred_perso.shape)

(753521, 5)


In [22]:
pred_perso = pred_perso.reset_index()
pred_perso = utils.getColumnValuesAsLists(pred_perso, "tag_pers_o_expected")
pred_perso.head()

,sentence_id,fold,token_id,sentence,tag_pers_o_expected,tag_pers_o_predicted
0,8,split0,233,James,[B-Masculine],B-Masculine
1,8,split0,234,Whyte,[I-Masculine],I-Unknown
2,8,split0,235,was,[O],O
3,8,split0,236,called,[O],O
4,8,split0,237,upon,[O],O


Remove `'O'` tags from the targets list since we are interested in the ability to apply the gendered and gender biased language related tags, and the `'O'` tags far outnumber the tags for gendered and gender biased language.

In [23]:
targets = list(clf_pers.classes_)
targets.remove('O')
print(targets)

['I-Unknown', 'I-Masculine', 'B-Masculine', 'I-Occupation', 'B-Occupation', 'B-Unknown', 'I-Feminine', 'B-Feminine']


In [19]:
# y_pred = clf_pers.predict(X_dev)

#### Evaluate: All Labels

In [20]:
# print("  - F1:", metrics.flat_f1_score(y_dev, y_pred, average="weighted", zero_division=0, labels=targets))
# print("  - Prec:", metrics.flat_precision_score(y_dev, y_pred, average="weighted", zero_division=0, labels=targets))
# print("  - Rec", metrics.flat_recall_score(y_dev, y_pred, average="weighted", zero_division=0, labels=targets))

  - F1: 0.42825014312210974
  - Prec: 0.4496106745338277
  - Rec 0.41771448419590135


Save the prediction data:

In [28]:
# df_dev_grouped = df_dev_grouped.rename(columns={"tag":"tag_pers_o_expected"})
# df_dev_grouped.insert(len(df_dev_grouped.columns), "tag_pers_o_predicted", y_pred)
# # df_dev_grouped.head()
# df_dev_grouped = df_dev_grouped.set_index("sentence_id")
# df_dev_exploded = df_dev_grouped.explode(list(df_dev_grouped.columns))
# df_dev_exploded.head()

In [2]:
# filename = "crf_{a}_pers_o_baseline_fastText{d}_nolingfeatures_predictions.csv".format(a=a, d=d)
# df_dev_exploded.to_csv(config.experiment1_output_path+filename)

<a id="eval"></a>
### Evaluation
#### Evaluate: Strict, Each Label

The built-in evaluation approach is strict, so unless the model predictions' labels are on text spans that exactly match the development data's test, the predicted labels will be deemed incorrect.

In [24]:
# a = "arow"
# category = "pers_o"
# filename = "crf_{a}_{c}_baseline_fastText{d}_nolingfeatures_predictions.csv".format(a=a, c=category, d=d)
# pred_perso = pd.read_csv(config.experiment1_output_path+filename)
# pred_perso = utils.getColumnValuesAsLists(pred_perso, "tag_{}_expected".format(category))
# # pred_pers.head()

Calculate performance metrics for each category of labels:

In [24]:
category = target_labels

In [25]:
pred_perso = utils.isPredictedInExpected(pred_perso, "tag_{}_expected".format(category), "tag_{}_predicted".format(category), '_merge', 'O')
pred_perso.head()

,sentence_id,fold,token_id,sentence,tag_pers_o_expected,tag_pers_o_predicted,_merge
0,8,split0,233,James,[B-Masculine],B-Masculine,true positive
1,8,split0,234,Whyte,[I-Masculine],I-Unknown,false positive
2,8,split0,235,was,[O],O,true negative
3,8,split0,236,called,[O],O,true negative
4,8,split0,237,upon,[O],O,true negative


Save the combined data:

In [26]:
filename = "crf_{a}_{t}_baseline_fastText{d}_nolingfeatures_evaluation.csv".format(a=a, t=target_labels, d=d)
pred_perso.to_csv(predictions_dir+filename)

In [27]:
pred_perso_stats = utils.getScoresByCatTags(
    pred_perso, "_merge", pers_o_label_subset[0], "tag_{}_expected".format(category), "tag_{}_predicted".format(category), "token_id"
)
for i in range(1, len(pers_o_label_subset)):
    tag_stats = utils.getScoresByCatTags(
        pred_perso, "_merge", pers_o_label_subset[i], "tag_{}_expected".format(category), "tag_{}_predicted".format(category), "token_id"
    )
    pred_perso_stats = pd.concat([pred_perso_stats, tag_stats])
pred_perso_stats

,tag(s),false negative,false positive,true positive,precision,recall,f1
0,B-Unknown,2337,2529,3496,0.580249,0.599349,0.589644
0,I-Unknown,4858,4434,6261,0.585414,0.563090,0.574035
0,B-Feminine,109,436,747,0.631445,0.872664,0.732712
0,I-Feminine,477,1024,1492,0.593005,0.757745,0.665329
0,B-Masculine,816,1101,1531,0.581687,0.652322,0.614983
0,I-Masculine,1631,2807,1886,0.401875,0.536252,0.459440
0,B-Occupation,1021,1016,1521,0.599527,0.598348,0.598937
0,I-Occupation,1626,1488,1538,0.508262,0.486094,0.496931


Save the statistics:

In [28]:
# pred_perso_stats.to_csv(
#     config.experiment1_agmt_path+"crf_{a}_baseline_fastText{d}_{c}_nolingfeatures_strict_agmt.csv".format(a=a, c=category, d=d)
# )
pred_perso_stats.to_csv(agreement_dir+"crf_{a}_baseline_fastText{d}_{c}_nolingfeatures_strict_agmt.csv".format(a=a, c=category, d=d))

### Annotation Agreement

Calculate agreement at the annotation level, so if the model labels any word correctly from a manually annotated text span, that annotation is recorded as being correctly labeled (`true positive`).  Note whether the models' labels are an `exact_match`, `label_match`, `category_match` or `mismatch`.

Load the annotation data:

*Note: `ann_id` of `9999` indicates no annotation*

In [28]:
# dev_df =  pd.read_csv(config.tokc_path+"experiment_input/token_train.csv", usecols=["sentence_id", "ann_id", "token_id", "tag"])
# dev_df.head()

Group the annotation data by token:

In [29]:
df_ann = perso_df[["sentence_id", "ann_id", "token_id", "tag"]] 
# ---------------------------
df_ann = utils.implodeDataFrame(df_ann, ["sentence_id", "ann_id", "token_id"])
df_ann = df_ann.reset_index()
# print(df_ann.shape)
# df_ann.head()

Align the columns of the dev and prediction DataFrames:

In [30]:
# Rename `sentence` column `token`
pred_perso = pred_perso.rename(columns={"sentence":"token"})
# pred_perso.head()

Join the data, adding the annotation IDs (`ann_id` column) to the prediction DataFrames:

In [31]:
index_list = ["sentence_id", "token_id"]

In [32]:
pred_perso_ann = pred_perso.join(df_ann.set_index(index_list), on=index_list, how="left")
pred_perso_ann = pred_perso_ann.drop(columns=["tag"])  # duplicate of tag_expected
assert pred_perso_ann.loc[pred_perso_ann["token_id"].isna()].shape[0] == 0
assert pred_perso_ann.loc[pred_perso_ann["ann_id"].isna()].shape[0] == 0
assert pred_perso_ann.loc[pred_perso_ann["tag_pers_o_predicted"].isna()].shape[0] == 0
assert pred_perso_ann.loc[pred_perso_ann["tag_pers_o_expected"].isna()].shape[0] == 0
# pred_perso_ann.head()

Explode the DataFrame:

In [33]:
pred_perso_ann = pred_perso_ann.explode(["tag_pers_o_expected"])

Generalize the BIO tags to label names:

In [34]:
# Get the predicted labels
pred_labels = list(pred_perso_ann["tag_{}_predicted".format(category)])
pred_labels = [label if label == "O" else label[2:] for label in pred_labels]
pred_perso_ann.insert(len(pred_perso_ann.columns), "label_{}_predicted".format(category), pred_labels)
# Get the lists of expected labels
exp_labels = list(pred_perso_ann["tag_{}_expected".format(category)])
exp_labels = [label if label == "O" else label[2:] for label in exp_labels]
pred_perso_ann.insert(len(pred_perso_ann.columns), "label_{}_expected".format(category), exp_labels)
# pred_perso_ann.head()

Group the data by annotation:

In [35]:
pred_perso_ann = pred_perso_ann.drop(columns=["tag_{}_expected".format(category), "tag_{}_predicted".format(category)])
pred_perso_ann = utils.implodeDataFrame(pred_perso_ann, ["sentence_id", "ann_id"])
pred_perso_ann = pred_perso_ann.reset_index()
pred_perso_ann.head()

,sentence_id,ann_id,fold,token_id,token,_merge,label_pers_o_predicted,label_pers_o_expected
0,0,99999,"[split4, split4, split4]","[0, 1, 2]","[Identifier, :, AA5]","[true negative, true negative, true negative]","[O, O, O]","[O, O, O]"
1,1,14384,"[split2, split2, split2, split2, split2, split...","[7, 7, 7, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10,...","[The, The, The, Very, Very, Very, Rev, Rev, Re...","[true positive, true positive, true positive, ...","[Masculine, Masculine, Masculine, Unknown, Unk...","[O, Unknown, Masculine, Unknown, Masculine, O,..."
2,1,24275,"[split2, split2, split2, split2, split2, split...","[7, 7, 7, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10,...","[The, The, The, Very, Very, Very, Rev, Rev, Re...","[true positive, true positive, true positive, ...","[Masculine, Masculine, Masculine, Unknown, Unk...","[O, Unknown, Masculine, Unknown, Masculine, O,..."
3,1,26233,"[split2, split2, split2, split2, split2, split...","[9, 9, 9, 9, 10, 10, 10, 10, 11, 11, 11, 11, 1...","[Rev, Rev, Rev, Rev, Prof, Prof, Prof, Prof, J...","[true positive, true positive, true positive, ...","[Unknown, Unknown, Unknown, Unknown, Unknown, ...","[Unknown, Masculine, Unknown, O, Unknown, O, M..."
4,1,52952,"[split2, split2, split2, split2, split2, split...","[7, 7, 7, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10,...","[The, The, The, Very, Very, Very, Rev, Rev, Re...","[true positive, true positive, true positive, ...","[Masculine, Masculine, Masculine, Unknown, Unk...","[O, Unknown, Masculine, Unknown, Masculine, O,..."


Record the agreements and disagreements:

In [36]:
agmt_types_perso, agmt_labels_perso = utils1.getAnnotationAgreement(pred_perso_ann, "label_pers_o_predicted", "label_pers_o_expected")

In [37]:
pred_perso_ann.insert(len(pred_perso_ann.columns), "annotation_agreement", agmt_types_perso)
pred_perso_ann.insert(len(pred_perso_ann.columns), "agreement_label", agmt_labels_perso)
pred_perso_ann.head()

,sentence_id,ann_id,fold,token_id,token,_merge,label_pers_o_predicted,label_pers_o_expected,annotation_agreement,agreement_label
0,0,99999,"[split4, split4, split4]","[0, 1, 2]","[Identifier, :, AA5]","[true negative, true negative, true negative]","[O, O, O]","[O, O, O]",true negative,O
1,1,14384,"[split2, split2, split2, split2, split2, split...","[7, 7, 7, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10,...","[The, The, The, Very, Very, Very, Rev, Rev, Re...","[true positive, true positive, true positive, ...","[Masculine, Masculine, Masculine, Unknown, Unk...","[O, Unknown, Masculine, Unknown, Masculine, O,...",true positive,Unknown
2,1,24275,"[split2, split2, split2, split2, split2, split...","[7, 7, 7, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10,...","[The, The, The, Very, Very, Very, Rev, Rev, Re...","[true positive, true positive, true positive, ...","[Masculine, Masculine, Masculine, Unknown, Unk...","[O, Unknown, Masculine, Unknown, Masculine, O,...",true positive,Unknown
3,1,26233,"[split2, split2, split2, split2, split2, split...","[9, 9, 9, 9, 10, 10, 10, 10, 11, 11, 11, 11, 1...","[Rev, Rev, Rev, Rev, Prof, Prof, Prof, Prof, J...","[true positive, true positive, true positive, ...","[Unknown, Unknown, Unknown, Unknown, Unknown, ...","[Unknown, Masculine, Unknown, O, Unknown, O, M...",true positive,Unknown
4,1,52952,"[split2, split2, split2, split2, split2, split...","[7, 7, 7, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10,...","[The, The, The, Very, Very, Very, Rev, Rev, Re...","[true positive, true positive, true positive, ...","[Masculine, Masculine, Masculine, Unknown, Unk...","[O, Unknown, Masculine, Unknown, Masculine, O,...",true positive,Unknown


In [38]:
metrics_perso_all = utils1.getAnnotationAgreementMetrics(pred_perso_ann, "all")
metrics_perso_pn = utils1.getAnnotationAgreementMetrics(pred_perso_ann.loc[~(pred_perso_ann.agreement_label.isin(["Occupation","O"]))], "Person Name")
metrics_perso_unk = utils1.getAnnotationAgreementMetrics(pred_perso_ann.loc[pred_perso_ann.agreement_label == "Unknown"], "Unknown")
metrics_perso_fem = utils1.getAnnotationAgreementMetrics(pred_perso_ann.loc[pred_perso_ann.agreement_label == "Feminine"], "Feminine")
metrics_perso_mas = utils1.getAnnotationAgreementMetrics(pred_perso_ann.loc[pred_perso_ann.agreement_label == "Masculine"], "Masculine")
metrics_perso_occ = utils1.getAnnotationAgreementMetrics(pred_perso_ann.loc[pred_perso_ann.agreement_label == "Occupation"], "Occupation")
metrics_perso = pd.concat([metrics_perso_all, metrics_perso_pn, metrics_perso_unk, metrics_perso_fem, metrics_perso_mas, metrics_perso_occ])
metrics_perso

,labels,false negative,true positive,false positive,precision,recall,f_1
0,all,10537,14834,6152,0.706852,0.584683,0.639990
0,Person Name,9045,12875,5199,0.712349,0.587363,0.643847
0,Unknown,5820,6783,2702,0.715129,0.538205,0.614180
0,Feminine,714,1912,746,0.719338,0.728104,0.723694
0,Masculine,2511,4180,1751,0.704772,0.624720,0.662336
0,Occupation,1492,1959,953,0.672734,0.567662,0.615747


Save the metrics and annotation-level data:

In [40]:
# metrics_perso.to_csv(
#     config.experiment1_agmt_path+"crf_{a}_baseline_fastText{d}_{c}_nolingfeatures_annot_agmt.csv".format(a=a, d=d, c=category)
# )
metrics_perso.to_csv(agreement_dir+"crf_{a}_baseline_fastText{d}_{c}_nolingfeatures_annot_agmt.csv".format(a=a, d=d, c=category))
pred_perso_ann[["sentence_id", "ann_id", "fold", "token_id", "annotation_agreement", "agreement_label"]].to_csv(
    predictions_dir+"crf_{a}_baseline_fastText{d}_{c}_nolingfeatures_annot_evaluation.csv".format(a=a, d=d, c=category)
)

### Loose Evaluation

As with the manual annotation evaluation, we want to evaluate the predictions more loosely, considering overlapping text spans in addition to exactly matching text spans.

#### Token Agreement

First, generalize the tokens' IOB tags to the label, and calculate agreement scores for each label.

In [41]:
pred_perso_labels = pred_perso.copy()
pred_perso_labels = pred_perso_labels.drop(columns=["_merge"])
tag_exp = list(pred_perso_labels["tag_{}_expected".format(category)])
tag_pred = list(pred_perso_labels["tag_{}_predicted".format(category)])
label_exp = [[tag if tag == "O" else tag[2:] for tag in tag_exp_list] for tag_exp_list in tag_exp]
label_pred = [tag if tag == "O" else tag[2:] for tag in tag_pred]
pred_perso_labels = pred_perso_labels.drop(columns=["tag_{}_expected".format(category), "tag_{}_predicted".format(category)])
pred_perso_labels.insert(len(pred_perso_labels.columns), "label_{}_expected".format(category), label_exp)
pred_perso_labels.insert(len(pred_perso_labels.columns), "label_{}_predicted".format(category), label_pred)
# pred_pers_labels.loc[pred_pers_labels.label_personname_predicted == "Feminine"].head()  # Looks good

Calculate the agreement metrics at the label level for each token:

In [42]:
tags = ['Unknown', 'Feminine', 'Masculine', 'Occupation']
pred_perso_labels = utils.isPredictedInExpected(pred_perso_labels, "label_{}_expected".format(category), "label_{}_predicted".format(category), '_merge', 'O')

pred_perso_stats = utils.getScoresByCatTags(
    pred_perso_labels, "_merge", tags[0], "label_{}_expected".format(category), "label_{}_predicted".format(category), "token_id"
)
for i in range(1, len(tags)):
    tag_stats = utils.getScoresByCatTags(
        pred_perso_labels, "_merge", tags[i], "label_{}_expected".format(category), "label_{}_predicted".format(category), "token_id"
    )
    pred_perso_stats = pd.concat([pred_perso_stats, tag_stats])
pred_perso_stats

,tag(s),false negative,false positive,true positive,precision,recall,f1
0,Unknown,7179,6197,10523,0.629366,0.594453,0.611411
0,Feminine,584,1264,2435,0.658286,0.806558,0.724918
0,Masculine,2438,3488,3837,0.523823,0.611474,0.564265
0,Occupation,2647,2287,3276,0.588891,0.553098,0.570434


Combine and save the performance measures:

In [43]:
# pred_perso_stats.to_csv(
#     config.experiment1_agmt_path+"crf_{a}_baseline_fastText{d}_{c}_nolingfeatures_loose_agmt.csv".format(a=a, d=d, c=category)
# )
pred_perso_stats.to_csv(agreement_dir+"crf_{a}_baseline_fastText{d}_{c}_nolingfeatures_loose_agmt.csv".format(a=a, d=d, c=category))

In [44]:
pred_perso_labels.head()

,sentence_id,fold,token_id,token,label_pers_o_expected,label_pers_o_predicted,_merge
0,8,split0,233,James,[Masculine],Masculine,true positive
1,8,split0,234,Whyte,[Masculine],Unknown,false positive
2,8,split0,235,was,[O],O,true negative
3,8,split0,236,called,[O],O,true negative
4,8,split0,237,upon,[O],O,true negative


Save the loose predictions (with labels instead of BIO tags):

In [45]:
pred_perso_labels.to_csv(predictions_dir+"crf_{a}_baseline_fastText{d}_{c}_nolingfeatures_loose_evaluation.csv".format(a=a, d=d, c=category))